# Head Pose Estimation Regrssion 

**In this Script, We will develop a python program that is able to predict The angels head postion (Yaw, Pitch, Roll) and draw the axies which represnt angels**

## Import Libraries

In [1]:
import mediapipe as mp
import cv2
import glob
import numpy as np
import csv
import os
from mat4py import loadmat
import scipy.io as sio
import math

In [2]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

## Get paths of images

In [3]:
paths,paths_mat=[],[]
# get the path/directory
folder_dir ="AFLW2000"

# iterate over files in
# that directory
for images in glob.iglob(f'{folder_dir}/*'):

    # check if the image ends with png
    if (images.endswith(".jpg")):
#         print(images)
        paths.append(images)

In [4]:
len(paths)

2000

## Get Landmarks and Save it in csv File

In [5]:
# For try
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic: 

    for idx, file in enumerate(paths):
        image = cv2.imread(file)
        image_height, image_width, _ = image.shape
        # Convert the BGR image to RGB before processing.
        results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
#         results = holistic.process(image)
#         print(results.face_landmarks)
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        
        break


In [6]:
results.face_landmarks.landmark[0]

x: 0.47754186391830444
y: 0.6920552253723145
z: -0.018991734832525253

In [7]:
num_coords = len(results.face_landmarks.landmark)
landmarks = []
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val)]
# landmarks
angles=['pitch','yaw','roll']
data = landmarks+angles

In [8]:
with open('landmarks_data.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(data)

In [9]:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    for idx, file in enumerate(paths):
        count=0
        image = cv2.imread(file)
        image_height, image_width, _ = image.shape
#         Convert the BGR image to RGB before processing.
        results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        try:
           
            face = results.face_landmarks.landmark
            NoseX = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].x * image_width
            NoseY=results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].y   *image_height
            LfeoX = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE_OUTER].x *image_width
            LfeoY = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE_OUTER].y* image_height
            
            dist = math.dist([NoseX, NoseY], [LfeoX, LfeoY])
        
            
            
            
            
            
            face_row = list(np.array([[   ((landmark.x *image_width)-NoseX)/dist, 
                                            ((landmark.y*image_height)-NoseY)/dist] for landmark in face]).flatten())
            
            random_file = file.split('\\')[-1].replace('.jpg' , '')
            mat_file = sio.loadmat('AFLW2000\\'+ random_file+ '.mat')
            pose_para = mat_file["Pose_Para"][0][:3]
            pitch = pose_para[0]
            yaw = pose_para[1]
            roll = pose_para[2]
            face_row.insert(((468*2)+0),pitch)
            face_row.insert(((468*2)+1),yaw)
            face_row.insert(((468*2)+2),roll)
            
            with open('landmarks_data.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(face_row) 
                
        except:
            pass
        

## Read and Prepare Date

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [11]:
df=pd.read_csv('landmarks_data.csv')
df.head()

,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,...,y465,x466,y466,x467,y467,x468,y468,pitch,yaw,roll
0,-0.159295,0.300003,-0.169120,0.067671,-0.146352,0.109180,-0.190952,-0.301066,-0.161892,-0.022124,...,-0.640003,0.132219,-0.604989,0.751723,-0.761274,0.807323,-0.806983,-0.399231,0.018227,0.085676
1,-0.334478,0.261430,-0.472663,-0.051103,-0.284650,0.053159,-0.358159,-0.332285,-0.480441,-0.137888,...,-0.525273,0.037087,-0.505745,0.562256,-0.565438,0.632099,-0.617696,0.470065,1.189533,0.300959
2,-0.981481,1.076963,-0.956155,0.954189,-0.976846,0.985318,-1.013231,0.778098,-0.954629,0.903266,...,0.634062,-0.919076,0.647000,-0.675460,0.588512,-0.651586,0.567591,-0.184650,0.881137,-0.236852
3,0.127447,0.433119,-0.086203,0.200610,0.035008,0.228611,-0.239562,-0.172509,-0.133581,0.104239,...,-0.642603,0.021166,-0.595805,0.629668,-0.972822,0.673793,-1.040024,-0.175379,0.299208,-0.373374
4,-0.078761,0.400796,-0.344684,0.090758,-0.104594,0.147864,-0.317465,-0.235742,-0.383646,-0.000681,...,-0.595326,-0.004369,-0.552671,0.427108,-0.790101,0.482372,-0.853066,-0.882169,1.198003,-1.033374


In [12]:
#Features
x=df.drop(['pitch','yaw','roll'], axis=1)
#Pitch label
yp=df['pitch']
#Yaw label
yy=df['yaw']
#roll label
yr=df['roll']
#labels
yall =df[['pitch','yaw','roll']]

In [13]:
#************************************************ PITCH ****************************************************************

X_train_p, X_val_p, y_train_p, y_val_p = train_test_split(x, yp, test_size=0.2, random_state=1234)

X_val_p, X_test_p, y_val_p, y_test_p = train_test_split(X_val_p, y_val_p, test_size=0.5, random_state=1234)


#************************************************* YAW ******************************************************************

X_train_y, X_val_y, y_train_y, y_val_y = train_test_split(x, yy, test_size=0.2, shuffle=True,random_state=1234)

X_val_y, X_test_y, y_val_y, y_test_y = train_test_split(X_val_y, y_val_y, test_size=0.5, shuffle=True,random_state=1234)


#*********************************************** Roll *******************************************************************

X_train_r, X_val_r, y_train_r, y_val_r = train_test_split(x, yr, test_size=0.2,shuffle=True, random_state=1234)

X_val_r, X_test_r, y_val_r, y_test_r = train_test_split(X_val_r, y_val_r, test_size=0.5,shuffle=True, random_state=1234)


#*************************************************** ALL ****************************************************************


X_train_all, X_val_all, y_train_all, y_val_all = train_test_split(x, yall, test_size=0.2,shuffle=True, random_state=1234)

X_val_all, X_test_all, y_val_all, y_test_all = train_test_split(X_val_all, y_val_all, test_size=0.5,shuffle=True, random_state=1234)



## Train Machine Learning Model

In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score 

## Model For Yaw

In [15]:
pipelines = {'rf_pca':make_pipeline(StandardScaler(),PCA(n_components=0.99), RandomForestRegressor())}



In [16]:
yaw_fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train_y, y_train_y)
    yaw_fit_models[algo] = model


In [17]:
Train_pred_yaw      = yaw_fit_models['rf_pca'].predict(X_train_y)
Validation_pred_yaw = yaw_fit_models['rf_pca'].predict(X_val_y)
yaw_hat=yaw_fit_models['rf_pca'].predict(X_test_y)

In [18]:
print("Train MSE "       , mean_squared_error( y_train_y,Train_pred_yaw ))
print("Validation MSE "  , mean_squared_error( y_val_y,Validation_pred_yaw ))

print("\n###########################\n")

print("Train r2_score "            , r2_score(y_train_y,Train_pred_yaw))
print("Validation r2_score "       , r2_score( y_val_y, Validation_pred_yaw))
print("Test r2_score "             , r2_score(y_test_y , yaw_hat))
print("\n###########################\n")

print("Train score "                   , yaw_fit_models['rf_pca'].score(X_train_y , y_train_y))
print("Validation Score "              , yaw_fit_models['rf_pca'].score(X_val_y, y_val_y))
print("test score: "                   ,yaw_fit_models ['rf_pca'].score(X_test_y, y_test_y))
 
print("\n###########################\n")

Train MSE  0.01047890341632861
Validation MSE  0.021080261527213606

###########################

Train r2_score  0.9686218335046136
Validation r2_score  0.9394623974504834
Test r2_score  0.9343837991459818

###########################

Train score  0.9686218335046136
Validation Score  0.9394623974504834
test score:  0.9343837991459818

###########################



In [19]:
import pickle 

with open('yaw_model.pkl', 'wb') as f:
    pickle.dump(yaw_fit_models ['rf_pca'], f)



## Model For Roll

In [20]:
pipelines = {'rfpcap' :make_pipeline(PCA(n_components=0.99), RandomForestRegressor(n_estimators=200))}

In [21]:
roll_fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train_r, y_train_r)
    roll_fit_models[algo] = model


In [22]:
Train_pred_roll     = roll_fit_models  ['rfpcap'].predict(X_train_r)
Validation_pred_roll = roll_fit_models ['rfpcap'].predict(X_val_r)
roll_hat             =roll_fit_models  ['rfpcap'].predict(X_test_r)

In [23]:
print("Train MSE "       , mean_squared_error( y_train_r,Train_pred_roll ))
print("Validation MSE "  , mean_squared_error( y_val_r,Validation_pred_roll ))

print("\n###########################\n")

print("Train r2_score "            , r2_score(y_train_r,Train_pred_roll))
print("Validation r2_score "       , r2_score( y_val_r, Validation_pred_roll))
print("Test r2_score "             , r2_score(y_test_r , roll_hat))
print("\n###########################\n")

print("Train score "                   , roll_fit_models['rfpcap'].score(X_train_r , y_train_r))
print("Validation Score "              , roll_fit_models['rfpcap'].score(X_val_r, y_val_r))
print("test score: "                   , roll_fit_models ['rfpcap'].score(X_test_r, y_test_r))
 
print("\n###########################\n")


Train MSE  0.06646012985469564
Validation MSE  0.017685983610097787

###########################

Train r2_score  0.8682355726980591
Validation r2_score  0.7661633815012057
Test r2_score  0.6912133596149082

###########################

Train score  0.8682355726980591
Validation Score  0.7661633815012057
test score:  0.6912133596149082

###########################



In [24]:
with open('roll_model.pkl', 'wb') as f:
    pickle.dump(roll_fit_models ['rfpcap'], f)

## Model For Pitch

In [25]:
pipelines = {'rf_pca':make_pipeline(StandardScaler(),PCA(n_components=0.99), RandomForestRegressor())}



In [26]:
pitch_fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train_p, y_train_p)
    pitch_fit_models[algo] = model


In [27]:
Train_pred_pitch     = pitch_fit_models['rf_pca'].predict(X_train_p)
Validation_pred_pitch = pitch_fit_models['rf_pca'].predict(X_val_p)
pitch_hat=pitch_fit_models['rf_pca'].predict(X_test_p)

In [28]:
#### print("Train MSE "       , mean_squared_error( y_train_y,Train_pred_yaw ))
print("Validation MSE "  , mean_squared_error( y_val_p,Validation_pred_pitch ))

print("\n###########################\n")

print("Train r2_score "            , r2_score(y_train_p,Train_pred_pitch))
print("Validation r2_score "       , r2_score( y_val_p, Validation_pred_pitch))
print("Test r2_score "             , r2_score(y_test_p , yaw_hat))
print("\n###########################\n")

print("Train score "                   , pitch_fit_models['rf_pca'].score(X_train_p , y_train_p))
print("Validation Score "              , pitch_fit_models['rf_pca'].score(X_val_p, y_val_p))
print("test score: "                   ,pitch_fit_models ['rf_pca'].score(X_test_p, y_test_p))
 
print("\n###########################\n")

Validation MSE  0.02620588682578975

###########################

Train r2_score  0.8487316056671604
Validation r2_score  0.5910949272804473
Test r2_score  -1.904695468085717

###########################

Train score  0.8487316056671604
Validation Score  0.5910949272804473
test score:  0.6016724515249767

###########################



In [29]:
with open('pitch_model.pkl', 'wb') as f:
    pickle.dump(pitch_fit_models ['rf_pca'], f)

## Try Models 

In [30]:
from math import cos ,sin

In [31]:
import pickle 
with open('yaw_model.pkl', 'rb') as f:
    yaw_model = pickle.load(f)

In [32]:
with open('pitch_model.pkl', 'rb') as f:
    pitch_model = pickle.load(f)

In [33]:
with open('roll_model.pkl', 'rb') as f:
    roll_model = pickle.load(f)

In [34]:
cap = cv2.VideoCapture(0)

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image_height, image_width, _ = image.shape

        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        
        
        
        
        try:
           
            face = results.face_landmarks.landmark
            NoseX = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].x * image_width
            NoseY=results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].y   *image_height
            LfeoX = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE_OUTER].x *image_width
            LfeoY = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE_OUTER].y* image_height
            
            dist = math.dist([NoseX, NoseY], [LfeoX, LfeoY])
            face_row = list(np.array([[   ((landmark.x *image_width)-NoseX)/dist, 
                                            ((landmark.y*image_height)-NoseY)/dist] for landmark in face]).reshape(1, -1))
            
            pitch = pitch_model.predict(face_row)
            yaw = yaw_model.predict(face_row)
            roll = roll_model.predict(face_row)
            

            yaw = -yaw
            tdx = NoseX
            tdy = NoseY
            size=100
    # X-Axis pointing to right. drawn in red
            x1 = size * (cos(yaw) * cos(roll)) + tdx
            y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green

            x2 = size * (-cos(yaw) * sin(roll)) + tdx
            y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
            x3 = size * (sin(yaw)) + tdx
            y3 = size * (-cos(yaw) * sin(pitch)) + tdy
            cv2.line(image, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
            cv2.line(image, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
            cv2.line(image, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

            
            

                
        except:
            pass


        
                        
        cv2.imshow('Raw Webcam Feed', image)
       
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450:

C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450:

C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450:

C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450:

C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarnin

...

C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450:

C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450:

C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450:

C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450:

C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Alhasan Gamal\anaconda3\lib\site-packages\sklearn\base.py:450: